In [1]:
import pandas as pd
from langdetect import detect
import string
import data_collector
import parser
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from collections import defaultdict
import pickle
import math
import numpy as np
import heapq

# 1. Data Collection

## Get the list of the books
We already have the list of books in the pc, so we won't do it again.

Set to `True` both dirs, bests and links parameters to create the correct directories and download the txt containing all the html links.

In [2]:
data_collector.download_books(dirs=False, bests=False, links=False)

## 1.1 Crawl books
We already have all the htmls in the pc, so we won't do it again.

Set to `True` both the books and fails parameters to download all the html pages and remove the ones with broken pages.

In [3]:
data_collector.download_books(books=False, fails=False)

## 1.2 Parse downloaded pages
Set to `True` the create parameter to parse the downloaded html pages and create the tsv file.

In [4]:
parser.create_tsv(create=False)

In [33]:
df = pd.read_csv('parsed_books.tsv', sep='\t')

In [6]:
df.shape

(29959, 12)

In [7]:
df.head()

,bookTitle,bookSeries,bookAuthors,ratingValue,ratingCount,reviewCount,Plot,numberOfPages,PublishingDate,Characters,Setting,Url
0,The Hunger Games,The Hunger Games #1,Suzanne Collins,4.33,6408798.0,172554.0,"Could you survive on your own in the wild, wit...",374.0,September 14th 2008,Katniss Everdeen Peeta Mellark Cato (Hunger Ga...,"District 12, Panem Capitol, Panem Panem",https://www.goodreads.com/book/show/2767052-th...
1,Harry Potter and the Order of the Phoenix,Harry Potter #5,J.K. Rowling,4.50,2525157.0,42734.0,There is a door at the end of a silent corrido...,870.0,September 2004,Sirius Black Draco Malfoy Ron Weasley Petunia ...,Hogwarts School of Witchcraft and Wizardry Lon...,https://www.goodreads.com/book/show/2.Harry_Po...
2,To Kill a Mockingbird,To Kill a Mockingbird,Harper Lee,4.28,4527405.0,91802.0,The unforgettable novel of a childhood in a sl...,324.0,May 23rd 2006,Scout Finch Atticus Finch Jem Finch Arthur Rad...,"Maycomb, Alabama",https://www.goodreads.com/book/show/2657.To_Ki...
3,Pride and Prejudice,NaN,Jane Austen,4.26,3017830.0,67811.0,Alternate cover edition of ISBN 9780679783268S...,279.0,October 10th 2000,Mr. Bennet Mrs. Bennet Jane Bennet Elizabeth B...,"United Kingdom Derbyshire, England England Her...",https://www.goodreads.com/book/show/1885.Pride...
4,Twilight,The Twilight Saga #1,Stephenie Meyer,3.60,4989910.0,104912.0,About three things I was absolutely positive.F...,501.0,September 6th 2006,Edward Cullen Jacob Black Laurent Renee Bella ...,"Forks, Washington Phoenix, Arizona Washington ...",https://www.goodreads.com/book/show/41865.Twil...


## 1.3 Dataset cleaning [preliminary steps]
Before actually jumping into the work itself, we want our dataframe to be clean, meaning that there are some preliminary steps we need to perform on it. First of all, missing data is something we should pay attention to. Lot's of rows are going to have missing data somewhere, and dealing with missing data it's not that nice. Notice that this will include different strategies for each of the column we will be considering (more details below). Then there is the problem with punctuation, stopwords, stems and so on so forth, so basic text data preprocessing. Let's make a brief recap:

1. **Missing data**
    - `bookTitle`: if a book is missing the title, then we can safely just remove the instance. In fact, books that are missing the title are actually missing all the informations, meaning that there is a problem with the GoodReads specific link. Also, even if a book was missing just the title, we wouldn't have a way to refer to it, thus it wouldn't be really useful considering we're building a search engine.
    - `bookSeries`, `Authors`, `Plot`, `PublishingDate`, `Characters`, `Setting`: if a book is missing one of the above mentioned columns, we can still include the book in the data, since the search engine could for example work with just the title. Obviously, we cannot just leave the values missing, since it would be really hard to perform any operation on that. These are all text columns, therefore the best way to address the missing values prolem is to replace NaNs with empty strings.
    - `ratingValue`, `NumberofPages`: TODO?
2. **Text data preprocessing**
    - Punctuation removal: this is the first step we want to perform, since it is going to make the next steps much easier (e.g., language detection will be easier if there aren't plots composed just by punctuation symbols).
    - Language detection: before doing anything else, we want to remove the books that present the books for which the plot isn't in english.
    - Stopwords removal (of the `Plot` column only)
    - Stemming (of the `Plot` column only)
    - Lowercase

### Missing values

#### Title
There are 774 books that are completely empty, and these corresponds to the ones that are missing the `bookTitle` column. If you give a look at the url, you can see that these are not given by our python script to download and parse the books, but actually from the fact that the link is broken. Also, you can see that all the books that are missing the `bookTitle` are also missing all the remaining data.

This means that we can safely just remove all the rows that are missing the `bookTitle` column.

In [34]:
n_missing = df[(df['bookTitle'].isna())].shape[0]
print('There are {} instances that are missing the `bookTitle` column.'.format(n_missing))
print()
df[(df['bookTitle'].isna())].head()

There are 774 instances that are missing the `bookTitle` column.



,bookTitle,bookSeries,bookAuthors,ratingValue,ratingCount,reviewCount,Plot,numberOfPages,PublishingDate,Characters,Setting,Url
311,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.goodreads.com/book/show/40937505\r\n
370,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.goodreads.com/book/show/30528535\r\n
379,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.goodreads.com/book/show/30528544\r\n
789,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.goodreads.com/book/show/40941582\r\n
1141,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.goodreads.com/book/show/5295735\r\n


In [35]:
# Remove empty books
df = df[(df['bookTitle'].notna())]

#### Text data

In [20]:
str_columns = ['bookSeries', 'bookAuthors', 'Plot', 'PublishingDate', 'Characters', 'Setting']

for col in str_columns:
    df[col] = df[col].fillna('')

### Text data preprocessing

#### Punctuation removal

**Observations**:

There are several ways to remove punctuations, including the use of exernal libraries (like nltk). But actually the fastest way to perform punctuation removal is the use of the internal methong translate, which is programmed in C and therefore it's much faster than the other options (give a look to this [link](https://stackoverflow.com/questions/265960/best-way-to-strip-punctuation-from-a-string) for a nice performance analysis of the various options).

In [18]:
def remove_punctuation(s):
    return s.translate(str.maketrans('', '', string.punctuation + '’—'))

In [21]:
for col in str_columns:
    df[col] = df[col].apply(remove_punctuation)

#### Language detection
There are four possibilities `Plot` column of a given book:
1. It is written in english
2. It is written in another language
3. It is empty
4. It contains symbols, numbers, and so on

We want to keep only the ones written in english or empty, so we are just going to discard the others.

In [40]:
def language(s):
    if s == '':
        return 'empty'
    try:
        return detect(s)
    except:
        return 'symbols'

In [41]:
df['plot_lang'] = df['Plot'].apply(language)

In [42]:
df = df[df['plot_lang'] == 'en'].drop(columns=['plot_lang'])

In [43]:
df.shape

(26122, 12)

#### Stopwords removal
We are not going to perform stopwords removal on all the columns, since we could remove important things (e.g., we don't want to remove anything from the names of the characters). The only column on which stopwords removal is necessary is `Plot`.

In [45]:
def remove_stopwords(s):
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(s)
    return ' '.join([w for w in tokens if w not in stop_words])

In [46]:
df['Plot'] = df['Plot'].apply(remove_stopwords)

#### Stemming
As for the stopwords removal, the only column on which stemming is necessary is `Plot`.

In [192]:
def stemming(s):
    ps = PorterStemmer()
    tokens = word_tokenize(s)
    return ' '.join([ps.stem(w) for w in tokens])

In [48]:
df['Plot'] = df['Plot'].apply(stemming)
df['Plot'] = df['Plot'].apply(stemming)

#### Lowercase
On the other hand, we want all the string columns to be lowercase, so that our search engine won't have problems with upper/lower case differences.

In [49]:
for col in str_columns:
    df[col] = df[col].apply(lambda w: w.lower())

In [50]:
df.head()

,bookTitle,bookSeries,bookAuthors,ratingValue,ratingCount,reviewCount,Plot,numberOfPages,PublishingDate,Characters,Setting,Url
0,The Hunger Games,the hunger games 1,suzanne collins,4.33,6408798.0,172554.0,could surviv wild everi one make sure dont liv...,374.0,september 14th 2008,katniss everdeen peeta mellark cato hunger gam...,district 12 panem capitol panem panem,https://www.goodreads.com/book/show/2767052-th...
1,Harry Potter and the Order of the Phoenix,harry potter 5,jk rowling,4.50,2525157.0,42734.0,there door end silent corridor and haunt harri...,870.0,september 2004,sirius black draco malfoy ron weasley petunia ...,hogwarts school of witchcraft and wizardry lon...,https://www.goodreads.com/book/show/2.Harry_Po...
2,To Kill a Mockingbird,to kill a mockingbird,harper lee,4.28,4527405.0,91802.0,the unforgett novel childhood sleepi southern ...,324.0,may 23rd 2006,scout finch atticus finch jem finch arthur rad...,maycomb alabama,https://www.goodreads.com/book/show/2657.To_Ki...
3,Pride and Prejudice,,jane austen,4.26,3017830.0,67811.0,altern cover edit isbn 9780679783268sinc immed...,279.0,october 10th 2000,mr bennet mrs bennet jane bennet elizabeth ben...,united kingdom derbyshire england england hert...,https://www.goodreads.com/book/show/1885.Pride...
4,Twilight,the twilight saga 1,stephenie meyer,3.60,4989910.0,104912.0,about three thing i absolut positivefirst edwa...,501.0,september 6th 2006,edward cullen jacob black laurent renee bella ...,forks washington phoenix arizona washington state,https://www.goodreads.com/book/show/41865.Twil...


In [51]:
df = df.reset_index(drop=True).reset_index()

### Save data

In [22]:
df.to_csv('clean_data.csv', index=False)

# 2. Search Engine

## 2.1 Conjunctive query

### 2.1.1 Create your index!

In [2]:
df = pd.read_csv('clean_data.csv')

In [3]:
# To save and load python dictionaries

def save_obj(obj, name):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [4]:
def term_index(documents):
    words = set()
    for s in documents:
        try:
            tokens = set(word_tokenize(s))
            words.update(tokens)
        except:
            continue
        
    term_index = {}
    for i, word in enumerate(words):
        term_index[word] = i
    return term_index

In [26]:
term_indexes = term_index(df['Plot'])

In [27]:
save_obj(term_indexes, 'term_index')

In [6]:
def inverted_index(documents, term_indexes):
    inv_index = defaultdict(list)
    for i, s in enumerate(documents):
        try:
            tokens = set(word_tokenize(s))
            for token in tokens:
                token_index = term_indexes[token]
                inv_index[token_index].append(i)
        except:
            continue
    return inv_index

In [29]:
inv_indexes = inverted_index(df['Plot'], term_indexes)

In [30]:
save_obj(inv_indexes, 'inverted_index')

### 2.1.2 Execute the query

In [5]:
term_indexes = load_obj('term_index')
inv_indexes = load_obj('inverted_index')
tfidf_indexes = load_obj('tfidf_index')

In [62]:
# Write it as a classs

class SimpleSearchEngine:
    def __init__(self, df, term_indexes, inv_indexes):
        self.df = df
        self.term_indexes = term_indexes
        self.inv_indexes = inv_indexes
        
    def execute_query(self, query):
        # Since we performed stemming on the plot column of the dataframe, we need to
        # perform stemming also on the query. Otherwise, our results wouldn't be accurate
        ps = PorterStemmer()
        query_tokens = set([ps.stem(w) for w in word_tokenize(query)])

        # Create term indexes for the query
        # notice: if one of the query element doesn't appear in the term_indexes dictionary
        # we can safely say that the **conjunctive** query has to return nothing
        term_indexes_tokens = []
        for token in query_tokens:
            if token in self.term_indexes.keys():
                term_indexes_tokens.append(self.term_indexes[token])
            else:
                print('No results')
                return pd.DataFrame(columns=['bookTitle', 'Plot', 'Url'])

        query_inv_indexes = {}
        for token_index in term_indexes_tokens:
            query_inv_indexes[token_index] = set(self.inv_indexes[token_index])

        # Since it is a conjuntive query, we need to intersect the results of each query token
        documents_id = sorted(set.intersection(*query_inv_indexes.values()))

        return self.df[self.df['index'].isin(documents_id)][['bookTitle', 'Plot', 'Url']]

In [69]:
simple_se = SimpleSearchEngine(df, term_indexes, inv_indexes)

In [64]:
s_engine.execute_query('survival games')

,bookTitle,Plot,Url
0,The Hunger Games,could surviv wild everi one make sure dont liv...,https://www.goodreads.com/book/show/2767052-th...
221,Catching Fire,spark are ignitingflam are spreadingand the ca...,https://www.goodreads.com/book/show/6148028-ca...
319,Mockingjay,the final book groundbreak hunger game trilog ...,https://www.goodreads.com/book/show/7260188-mo...
337,Legend,what western unit state home republ nation per...,https://www.goodreads.com/book/show/9275658-le...
652,The Magus,thi dare literari thriller rich erot suspen on...,https://www.goodreads.com/book/show/16286.The_...
...,...,...,...
25063,The Manhattan Hunt Club,in manhattan hunt club john saul plumb depth m...,https://www.goodreads.com/book/show/6553.The_M...
25527,Love's Forbidden Flower,plea note thi new adult romanc novel involv tw...,https://www.goodreads.com/book/show/16189423-l...
25802,The Southpaw,the southpaw stori come age america way baseb ...,https://www.goodreads.com/book/show/413736.The...
25823,Devil's Own,after surviv slaveri aiden macalpin noth thoug...,https://www.goodreads.com/book/show/8705483-de...


## 2.2 Conjunctive query & Ranking score

### 2.2.1 Inverted index

In [33]:
def tfidf_inv_indexes(documents, term_indexes, inv_indexes, corpus=df):
    tfidf_indexes = defaultdict(dict)
    for i, s in enumerate(documents):
        try:
            tokens = word_tokenize(s)
            tokens_set = set(tokens)
            n_tokens = len(tokens)
            norm = 0
            for token in tokens_set:
                token_index = term_indexes[token]
                tf = s.count(token) / n_tokens
                idf = math.log10(len(corpus) / (len(inv_indexes[token_index])))
                tf_idf = tf * idf
                norm += tf_idf**2
                tfidf_indexes[token_index][i] = tf * idf
            
            # Normalize each document tfidf
            norm = np.sqrt(norm)
            for token in tokens_set:
                token_index = term_indexes[token]
                tfidf_indexes[token_index][i] /= norm
        except:
            continue
    return tfidf_indexes

In [24]:
tfidf_indexes = tfidf_inv_indexes(df['Plot'], term_indexes, inv_indexes)

In [49]:
save_obj(tfidf_indexes, 'tfidf_index')

### 2.2.2 Execute the query

In [77]:
class TfidfSearchEngine:
    def __init__(self, df, term_indexes, inv_indexes, tfidf_indexes, simple_se):
        self.df = df
        self.term_indexes = term_indexes
        self.inv_indexes = inv_indexes
        self.tfidf_indexes = tfidf_indexes
        self.simple_se = simple_se
        
    def execute_query(self, query, k):
        ps = PorterStemmer()
        query_tokens = set([ps.stem(w) for w in word_tokenize(query)])

        tokens_ids = []
        for token in query_tokens:
            try:
                tokens_ids.append(self.term_indexes[token])
            except:
                return

        conj_query = self.simple_se.execute_query(query)

        conj_query_ids = conj_query.index


        scores = np.array([])
        for doc in conj_query_ids:
            tfidf = 0
            for token_id in tokens_ids:
                tfidf += self.tfidf_indexes[token_id][doc]

            scores = np.append(scores, tfidf)
            
        conj_query['Similarity'] = scores

        # Use heaps to extract top k rows
        conj_query_list = conj_query.values.tolist()
        heapq.heapify(conj_query_list)
        max_k = heapq.nlargest(k, conj_query_list, key = lambda t: t[3])

        max_k_df = pd.DataFrame(data=max_k, columns=['bookTitle', 'Plot', 'Url', 'Similarity'])

        return max_k_df

In [78]:
tfidf_se = TfidfSearchEngine(df, term_indexes, inv_indexes, tfidf_indexes, simple_se)

In [80]:
tfidf_se.execute_query('wizard', 10)

,bookTitle,Plot,Url,Similarity
0,So You Want to Be a Wizard,nita callahan end rope bulli whove hound schoo...,https://www.goodreads.com/book/show/116563.So_...,0.487815
1,Sourcery,when last seen singularli inept wizard rincewi...,https://www.goodreads.com/book/show/34499.Sour...,0.434745
2,The Wizard of Dark Street,oona crate born wizard apprent anoth destini m...,https://www.goodreads.com/book/show/9668661-th...,0.388242
3,"Fairy Tail, Vol. 01",the wick side of wizardrycelesti wizard luci w...,https://www.goodreads.com/book/show/2454986.Fa...,0.358335
4,Wizards at War,nita kit return wizardli holiday look forward ...,https://www.goodreads.com/book/show/116567.Wiz...,0.339397
5,Mélusine,mélusin citi secret lie pleasur pain magic co...,https://www.goodreads.com/book/show/492069.M_l...,0.331146
6,The Young Wizards,book club collect so you want be wizard deep w...,https://www.goodreads.com/book/show/2175442.Th...,0.314714
7,Proven Guilty,there love lost harri dresden wizard chicago p...,https://www.goodreads.com/book/show/91474.Prov...,0.312884
8,Harry Potter and the Goblet of Fire,harri potter midway train wizard come age harr...,https://www.goodreads.com/book/show/6.Harry_Po...,0.301307
9,Fantastic Beasts - The Crimes of Grindelwald: ...,at end fantast beast where find them power dar...,https://www.goodreads.com/book/show/39330961-f...,0.298715
